In [2]:
import os
import sys
sys.path.append(os.environ.get('NOTEBOOK_ROOT'))

import xarray as xr
import numpy as np
import pandas as pd

# Ensure string casts of NumPy arrays 
# print as much as possible (no '...').
np.set_printoptions(threshold=sys.maxsize)

import matplotlib.pyplot as plt

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-zdoi8kvc because the default path (/home/jovyan/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [3]:
from odc_gee.earthengine import Datacube as GEE_Datacube
dc = GEE_Datacube()

In [4]:
dc.list_products()

,name,description,format,creation_time,product_family,platform,time,lon,instrument,region_code,label,lat,dataset_maturity,crs,resolution,tile_size,spatial_dimensions
id,,,,,,,,,,,,,,,,,
2,ls8_l2_c1_t1_google,Landsat 8 Collection 1 Level 2 Surface Reflect...,None,None,None,None,None,None,None,None,None,None,None,EPSG:4326,"(-0.00027777777778, 0.00027777777778)",None,"(latitude, longitude)"
1,ls8_l2_c1_t2_google,Landsat 8 Collection 1 Level 2 Surface Reflect...,None,None,None,None,None,None,None,None,None,None,None,EPSG:4326,"(-0.00027777777778, 0.00027777777778)",None,"(latitude, longitude)"


In [5]:
ds = dc.load(product='ls8_l2_c1_t1_google',
             lat=(36.894872, 36.969353),
             lon=(-76.394949,-76.260025),
             time=('2014-01-01', '2014-06-30'),
             measurements=['pixel_qa'])

In [17]:
from utils.data_cube_utilities.clean_mask import landsat_qa_clean_mask

clean_da = \
    landsat_qa_clean_mask(ds, 'LANDSAT_8', collection='c1', level='l2')

In [14]:
import os

from jinja2 import Template
from IPython.display import HTML

import psutil

from utils.data_cube_utilities.dc_time import _n64_to_datetime, dt_to_str

VOXEL_VIS_WEB_SVR_CMD = 'python3 server.py &'

class VoxelVisualizer():
    def _launch_server_if_not_running(self):
        # Determine if the server is running.
        process_cmds = (p.cmdline() for p in psutil.process_iter())
        cmd_found = False
        for cmd in process_cmds:
            for token in VOXEL_VIS_WEB_SVR_CMD.split():
                if token != '&' and token not in cmd:
                    break
                cmd_found = True
                break
            if cmd_found:
                break
        # If the server is not running, start it.
        if not cmd_found:
            os.system(VOXEL_VIS_WEB_SVR_CMD)
    
    def __init__(self):
        # Launch the webserver.    
        self._launch_server_if_not_running()
        fs = open('template.html','r')
        self.template = Template(fs.read())
        fs.close()
    
#     def draw(self):
#         if not da.dtype == 'boolean':
#             raise Exception("You need to pass a boolean xarray.DataArray to use this.")
        
#         filled_template = self.template.render() #data = str_rep)
#         self.save(filled_template, 'out1.html')
#         vox_vis_server_port = os.environ['VOXEL_VISUALIZER_PORT']
#         self.iframe = HTML(f"""
#         <iframe id='iframe' width=600, height=350></iframe>
#         <script>
#           var hostname = window.location.hostname;
#           var url = 'http://' + hostname + ':{vox_vis_server_port}/static/out1.html'
#           console.log(url);
#           document.getElementById('iframe').src = url;
#         </script>""")
#         return self.iframe

    def draw(self, da: xr.DataArray):
#         if not np.array_equal(arr, arr.astype(bool)):
#             raise Exception("You need to pass a boolean array to use this")
        if not da.dtype == 'bool':
            raise Exception("You need to pass a boolean xarray.DataArray to use this.")
        
        da = da.astype(np.int8)
        
        da_str = str(da.values.tolist())#.replace('array(', '').replace(')', '')#.replace('\n', ',').replace(',,', ',')
#         print(f'da_str: {da_str[-1000:]}')
#         times_str = da.time.values
        times_str = str([dt_to_str(_n64_to_datetime(time), fmt='%Y-%m-%dT%H:%M:%S.%f') 
                         for time in clean_da.time.values]).replace(',', ',\n')
#         print(f'times_str: {times_str}')
        filled_template = self.template.render(data_array=da_str, times=times_str)
        self.save(filled_template, 'out1.html')
        vox_vis_server_port = os.environ['VOXEL_VISUALIZER_PORT']
        self.iframe = HTML(f"""
        <iframe id='iframe' width=600, height=350></iframe>
        <script>
          var hostname = window.location.hostname;
          var url = 'http://' + hostname + ':{vox_vis_server_port}/static/out1.html'
          console.log(url);
          document.getElementById('iframe').src = url;
        </script>""")
        return self.iframe

    def save(self, data, location):
        fs = open(location, "w")
        fs.write(data)
        fs.close()
        return 

In [19]:
clean_da.dtype == 'bool'

True

In [15]:
voxel_visualizer = VoxelVisualizer()

In [16]:
voxel_visualizer.draw(clean_da)